<a href="https://colab.research.google.com/github/silveira-alysson/Strategic-Intelligence-LLM/blob/main/ParseWhole10Kv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import os
import unicodedata
from multiprocessing import Pool, cpu_count
import pickle
import nltk
from nltk import tokenize
nltk.download('punkt')

def parse10k(file):
  temp_dict = dict()
  try:
    soup = BeautifulSoup(open(file), 'lxml')
    for table in soup.find_all("table"):
        table.extract()
    for filing_document in soup.find_all(
            'document'):  # The document tags contain the various components of the total 10K filing pack

        # The 'type' tag contains the document type
        document_type = filing_document.type.find(text=True, recursive=False).strip()


        if document_type == "10-K":  # Once the 10K text body is found

            # Grab and store the 10K text body

            TenKtext = filing_document.find('text').extract().text
            TenKtext = unicodedata.normalize("NFKD", TenKtext)

            # Clean newly extracted text
            TenKItem = TenKtext.strip()  # Remove starting/ending white spaces
            TenKItem = TenKItem.replace('\n', ' ')  # Replace \n (new line) with space
            TenKItem = TenKItem.replace('\r', '')  # Replace \r (carriage returns-if you're on windows) with space
            TenKItem = TenKItem.replace(' ', ' ')  # Replace " " (a special character for space in HTML) with space
            TenKItem = TenKItem.replace(' ', ' ')  # Replace " " (a special character for space in HTML) with space
            while '  ' in TenKItem:
                TenKItem = TenKItem.replace('  ', ' ')  # Remove extra spaces
            namef = file[-24:-4]
            #temp_dict = {file[-24:-4]:tokenize.sent_tokenize(TenKItem)}
  except:
    TenKItem = ""
    namef = file[-24:-4]
    #temp_dict[file[-24:-4]]= ""
  return [namef, TenKItem]

if __name__ == '__main__':

  files = []
  sent_corpus = dict()

  input_path = '/content/drive/MyDrive/AssetPricingPaper/Inputs/'
  output_path = '/content/drive/MyDrive/AssetPricingPaper/Outputs/'
  files = [os.path.join(input_path,filename) for filename in os.listdir(input_path)]
  processes = cpu_count()
  chunk_size = int(len(files) // (processes**2) + 1)
  pool = Pool(processes)
  results = pool.imap_unordered(parse10k, files)
  for result in results:
    filename = os.path.join(output_path, result[0])
    with(open(filename+".txt", "w")) as f:
      f.write(result[1])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
